<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/%5BBasics%5D_PyTorch_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Basics

### 1.1 PyTorch Basics

In [23]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
'''
1. torchvision.transforms是PyTorch中的图像预处理包，包含了很多种对图像数据进行变换的函数
'''
import torchvision.transforms as transforms


In [4]:
# Create tensors
x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)
'''
1. requires_grad是PyTorch中Tensor的一个属性，用来说明当前量是否需要在计算中保留对应的梯度信息，
如果requires_grad = False，则梯度无法回传，训练失败
'''

# Build a computational graph
y = w * x + b # y = 2 * x + 3

# Compute gradients
y.backward()

# Print out the gradients
'''
我们之前定义的函数为 y = w * x + b
（1）对x求梯度结果为其系数w, 即x.grad = w = 2
（2）对w求梯度结果为其系数x, 即w.grad = x = 1
（3）对b求梯度结果为其系数1, 即b.grad = 1
'''
print(x.grad) # x.grad = 2
print(w.grad) # w.grad = 1
print(b.grad) # b.grad = 1

tensor(2.)
tensor(1.)
tensor(1.)


In [14]:
# Create tensors of shape (10, 3) and (10, 2)
'''
torch.randn : Returns a tensor filled with random numbers from a normal distribution
with mean `0` and variance `1` (also called the standard normal
distribution).
'''
x = torch.randn(10, 3) 
y = torch.randn(10, 2)

# Build a fully connected layer
linear = nn.Linear(3, 2) # matrix (10,3) x (3,2) = (10,2) => w * x + b => y
print('w: ', linear.weight)
print('b: ', linear.bias)

# Build loss function and optimizer
criterion = nn.MSELoss() # Loss Function
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.01)
'''
optimizer优化器的参数非常直观：
(1) 待优化model的所有参数parameters()
(2) 学习率learning_rate
'''

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print('loss: ', loss.item())
'''
The item() method extracts the loss’s value as a Python float.
'''

# Backward pass
loss.backward()

# Print out the gradients
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

# 1-step gradient descent
optimizer.step()
'''
`optimizer.step` performs a parameter update based on the [current gradient]
(which stored in `.grad` attribute of a parameter) and the update rule
'''

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[-0.4947, -0.4644,  0.2885],
        [ 0.5699, -0.3475, -0.1600]], requires_grad=True)
b:  Parameter containing:
tensor([-0.0283,  0.4911], requires_grad=True)
loss:  1.128941297531128
dL/dw:  tensor([[-0.5392, -0.1337, -0.4095],
        [ 0.2882, -0.2899,  0.1438]])
dL/db:  tensor([-0.0459,  0.8059])
loss after 1 step optimization:  1.1158812046051025


In [15]:
# Loading data from numpy

# Create a numpy array
x = np.array([[1, 2], [3, 4]])

# Numpy array => Torch tensor
y = torch.from_numpy(x)

# Torch tensor => Numpy array
z = y.numpy()

In [27]:
# Input pipeline
import torchvision.transforms as transforms
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True, 
                                             transform = transforms.ToTensor(),
                                             download = True)

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size(), label)

# Dataloader (this provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = 64,
                                           shuffle = True)

# When iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)
'''
Get an iterator from an object.  In the first form, the argument must
supply its own iterator, or be a sequence.
'''

# Mini-batch images and labels
images, labels = data_iter.next()

# Actual usage of the dataloader is as below
for images, labels in train_loader:
  # [Training code should be written here]
  pass

Files already downloaded and verified
torch.Size([3, 32, 32]) 6


In [29]:
'''
How to build custom dataset
'''

# You should build your custom dataset as below
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):
    # 1. Initialize file paths or a list of file names.
    pass
  
  def _getitem__(self, index):
    # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open)
    # 2. Preprocess the data (e.g. torchvision.Transform)
    # 3. Return a data pair (e.g. image and label)
    pass

  def __len__(self):
    # You should change 100 to the [total size] of your dataset
    return 100

# You can then use the prebuilt data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset = custom_dataset,
                                           batch_size = 64,
                                           shuffle = True)

In [30]:
'''
Using pretrained model
'''

# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained = True)

# If you want to finetune only the top layer of the model, set as below
for param in resnet.parameters():
  param.requires_grad = False
'''
1. 通过设置最顶层的参数的required_grad = False，这样梯度回传就会在这里停止
2. 这样我们就可以只针对顶层的参数进行调参
'''

# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

# Forward pass.
images = torch.randn(64, 3, 224, 224) # 64 images, 3 channels, 224 * 224 
outputs = resnet(images)
print(outputs.size()) # (64, 100)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



torch.Size([64, 100])


In [31]:
'''
Save and load the model
'''
# Save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
'''
因为有的时候Model会比较大，我们倾向于只读取Model的参数
'''
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>